In [52]:
import pandas as pd
import numpy as np
import importlib
import utilities.utility as utility
from pypfopt import EfficientFrontier, risk_models, expected_returns

import utilities.variables as variables
import utilities.api_ticker_service as data_ticker_service
importlib.reload(data_ticker_service)
importlib.reload(variables)
importlib.reload(utility)

<module 'utilities.utility' from '/Users/herbishtini/Documents/UNI/Master Thesis/sustainability_portfolio_optimisation/utilities/utility.py'>

### Average historical return

In [53]:
df_monthly_adj_close = pd.read_csv('../../data/df_monthly_adjacent_close.csv', index_col=0)

#### Converting 'Adj Close' Values to Percentage Change
The "Adj Close" will be transformed to represent the percentage change from the previous day's close. The percentage change will be calculated as 1 + percentage change, which indicates the relative change in the adjusted close values day-over-day.

In [54]:
df_monthly_return = df_monthly_adj_close.copy(deep=True)
for column in df_monthly_return:
    if column != 'Date':
        df_monthly_return[column] = 1 + df_monthly_adj_close[column].pct_change().round(2)
        df_monthly_return[column].dropna()

In [55]:
df_monthly_return.to_csv('../../data/df_monthly_returns_complete.csv')

### Return rate

In [56]:
df_monthly_return = pd.read_csv('../../data/df_monthly_returns_complete.csv', index_col=0)
df_overview = pd.read_csv('../../data/intermediate/data_5_scaled.csv', index_col=0)

In [57]:
df_overview

,company_name,company_esg_score,company_esg_score_group,industry,stock_exchange,stock_ticker_symbol,market_capital,market_capital_euro,trailing_pe,beta,return_on_equity,earnings_growth,company_esg_score_scale,market_capital_scale,score
0,RS Group Plc,4.5,Negligible ESG Risk,Technology Hardware,LON,RS1.L,3.668791e+09,4.365861e+09,19.858974,0.863,0.13226,-0.368,0.000000,0.000273,0.000205
1,"Kimball Electronics, Inc.",4.5,Negligible ESG Risk,Technology Hardware,NAS,KE,4.281351e+08,3.938843e+08,21.370369,1.258,0.03854,NaN,0.000000,0.000024,0.000018
2,TAG Immobilien AG,4.6,Negligible ESG Risk,Real Estate,ETR,TEG.DE,2.765612e+09,2.765612e+09,NaN,1.037,-0.03766,NaN,0.006452,0.000173,0.001743
3,LEG Immobilien SE,5.1,Negligible ESG Risk,Real Estate,ETR,LEG.DE,6.927171e+09,6.927171e+09,NaN,0.957,-0.08126,NaN,0.038710,0.000434,0.010003
4,"Steelcase, Inc.",5.3,Negligible ESG Risk,Commercial Services,NYS,SCS,1.492860e+09,1.373431e+09,17.210526,1.314,0.10633,8.000,0.051613,0.000086,0.012968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1657,"KVH Industries, Inc. (Delaware)",20.0,Low ESG Risk,Technology Hardware,NAS,KVHI,9.127629e+07,9.127629e+07,NaN,0.594,-0.14536,NaN,1.000000,0.000006,0.250004
1658,Moonpig Group Plc,20.0,Low ESG Risk,Retailing,LON,MOON.L,7.018797e+08,7.018797e+08,2035.000000,1.238,NaN,-0.072,1.000000,0.000044,0.250033
1659,"NeoGenomics, Inc.",20.0,Low ESG Risk,Pharmaceuticals,NAS,NEO,1.940312e+09,1.940312e+09,NaN,1.191,-0.08391,NaN,1.000000,0.000121,0.250091
1660,"Japan Material Co., Ltd.",20.0,Low ESG Risk,Semiconductors,TKS,6055.T,1.704423e+11,1.704423e+11,27.933996,0.585,0.13585,0.379,1.000000,0.010681,0.258011


In [58]:
# move Date index to a new column
df_monthly_adj_close = df_monthly_adj_close.reset_index(level=0)
df_monthly_return = df_monthly_return.reset_index(level=0)

# Creates a new column for each year group (1, 5, 10, 25 year returns)
utility.set_yearly_return_rates_by_years(df_overview, df_monthly_return)
df_overview

,company_name,company_esg_score,company_esg_score_group,industry,stock_exchange,stock_ticker_symbol,market_capital,market_capital_euro,trailing_pe,beta,return_on_equity,earnings_growth,company_esg_score_scale,market_capital_scale,score,return_rate_1y_avg,return_rate_5y_avg,return_rate_10y_avg,return_rate_25y_avg
0,RS Group Plc,4.5,Negligible ESG Risk,Technology Hardware,LON,RS1.L,3.668791e+09,4.365861e+09,19.858974,0.863,0.13226,-0.368,0.000000,0.000273,0.000205,1.065700,1.054736,1.175506,1.051726
1,"Kimball Electronics, Inc.",4.5,Negligible ESG Risk,Technology Hardware,NAS,KE,4.281351e+08,3.938843e+08,21.370369,1.258,0.03854,NaN,0.000000,0.000024,0.000018,0.717103,1.009610,1.045201,1.017841
2,TAG Immobilien AG,4.6,Negligible ESG Risk,Real Estate,ETR,TEG.DE,2.765612e+09,2.765612e+09,NaN,1.037,-0.03766,NaN,0.006452,0.000173,0.001743,1.284466,0.980213,1.102412,0.969896
3,LEG Immobilien SE,5.1,Negligible ESG Risk,Real Estate,ETR,LEG.DE,6.927171e+09,6.927171e+09,NaN,0.957,-0.08126,NaN,0.038710,0.000434,0.010003,1.394756,1.013014,1.079424,1.048932
4,"Steelcase, Inc.",5.3,Negligible ESG Risk,Commercial Services,NYS,SCS,1.492860e+09,1.373431e+09,17.210526,1.314,0.10633,8.000,0.051613,0.000086,0.012968,1.088965,0.979103,1.008675,1.036085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1657,"KVH Industries, Inc. (Delaware)",20.0,Low ESG Risk,Technology Hardware,NAS,KVHI,9.127629e+07,9.127629e+07,NaN,0.594,-0.14536,NaN,1.000000,0.000006,0.250004,0.940916,0.828128,0.904893,1.018153
1658,Moonpig Group Plc,20.0,Low ESG Risk,Retailing,LON,MOON.L,7.018797e+08,7.018797e+08,2035.000000,1.238,NaN,-0.072,1.000000,0.000044,0.250033,1.152179,0.851273,0.922644,0.968308
1659,"NeoGenomics, Inc.",20.0,Low ESG Risk,Pharmaceuticals,NAS,NEO,1.940312e+09,1.940312e+09,NaN,1.191,-0.08391,NaN,1.000000,0.000121,0.250091,0.865053,0.904706,1.143484,1.158677
1660,"Japan Material Co., Ltd.",20.0,Low ESG Risk,Semiconductors,TKS,6055.T,1.704423e+11,1.704423e+11,27.933996,0.585,0.13585,0.379,1.000000,0.010681,0.258011,0.652273,1.026089,1.202058,1.150042


### Volatility
To calculate the volatility of a stock's monthly returns, there are several steps that should be applied to the historical price data.
* Calculate Monthly Returns
* Calculate the Average Monthly Return
* Calculate the Monthly Return Deviations
* Square the Deviations
* Calculate the Variance
* Calculate the Standard Deviation

In [59]:
utility.set_volatility_by_years(df_overview, df_monthly_adj_close)

In [60]:
df_overview

,company_name,company_esg_score,company_esg_score_group,industry,stock_exchange,stock_ticker_symbol,market_capital,market_capital_euro,trailing_pe,beta,...,market_capital_scale,score,return_rate_1y_avg,return_rate_5y_avg,return_rate_10y_avg,return_rate_25y_avg,volatility_1y,volatility_5y,volatility_10y,volatility_25y
0,RS Group Plc,4.5,Negligible ESG Risk,Technology Hardware,LON,RS1.L,3.668791e+09,4.365861e+09,19.858974,0.863,...,0.000273,0.000205,1.065700,1.054736,1.175506,1.051726,0.074644,0.079067,0.080528,0.070054
1,"Kimball Electronics, Inc.",4.5,Negligible ESG Risk,Technology Hardware,NAS,KE,4.281351e+08,3.938843e+08,21.370369,1.258,...,0.000024,0.000018,0.717103,1.009610,1.045201,1.017841,0.094499,0.124539,0.109874,0.109874
2,TAG Immobilien AG,4.6,Negligible ESG Risk,Real Estate,ETR,TEG.DE,2.765612e+09,2.765612e+09,NaN,1.037,...,0.000173,0.001743,1.284466,0.980213,1.102412,0.969896,0.079766,0.111259,0.087658,0.107509
3,LEG Immobilien SE,5.1,Negligible ESG Risk,Real Estate,ETR,LEG.DE,6.927171e+09,6.927171e+09,NaN,0.957,...,0.000434,0.010003,1.394756,1.013014,1.079424,1.048932,0.083469,0.094127,0.076087,0.073152
4,"Steelcase, Inc.",5.3,Negligible ESG Risk,Commercial Services,NYS,SCS,1.492860e+09,1.373431e+09,17.210526,1.314,...,0.000086,0.012968,1.088965,0.979103,1.008675,1.036085,0.088897,0.119393,0.109562,0.106332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1657,"KVH Industries, Inc. (Delaware)",20.0,Low ESG Risk,Technology Hardware,NAS,KVHI,9.127629e+07,9.127629e+07,NaN,0.594,...,0.000006,0.250004,0.940916,0.828128,0.904893,1.018153,0.062427,0.095238,0.098379,0.143359
1658,Moonpig Group Plc,20.0,Low ESG Risk,Retailing,LON,MOON.L,7.018797e+08,7.018797e+08,2035.000000,1.238,...,0.000044,0.250033,1.152179,0.851273,0.922644,0.968308,0.089974,0.127531,0.127531,0.127531
1659,"NeoGenomics, Inc.",20.0,Low ESG Risk,Pharmaceuticals,NAS,NEO,1.940312e+09,1.940312e+09,NaN,1.191,...,0.000121,0.250091,0.865053,0.904706,1.143484,1.158677,0.116134,0.169280,0.145000,0.170531
1660,"Japan Material Co., Ltd.",20.0,Low ESG Risk,Semiconductors,TKS,6055.T,1.704423e+11,1.704423e+11,27.933996,0.585,...,0.010681,0.258011,0.652273,1.026089,1.202058,1.150042,0.083176,0.102194,0.110772,0.116846


### Store changes

In [61]:
df_overview.to_csv('../../data/df_overview.csv')
df_monthly_return.to_csv('../../data/df_monthly_returns_complete.csv')